# try question answer system

In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
from pinecone import Pinecone

In [2]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [3]:
embeddings = download_hugging_face_embeddings()

In [4]:
PINECONE_API_KEY = "bb77b29f-7fe6-4390-9510-cec577216f50"
pc = Pinecone(api_key=PINECONE_API_KEY)  

index_name="medical-chatbot"
index = pc.Index(index_name)

In [5]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [17]:
def retriver_results(query, top_res):
    # create the query vector
    xq = embeddings.embed_query(query)
    # now query
    xc = index.query(vector=xq, top_k=top_res, include_metadata=True, namespace="real", include_values=True)

    contexts = []
    contexts = contexts + [
            x['metadata']['text'] for x in xc['matches']
        ]
    
    querydocs = []
    from langchain.docstore.document import Document

    for eachcont in contexts:
        doc =  Document(page_content=eachcont, metadata={"source": "local"})
        querydocs.append(doc)

    return querydocs, "".join(contexts)

In [7]:
from langchain.chains.question_answering import load_qa_chain
chain=load_qa_chain(llm,chain_type="stuff")

# pass result chunk after similarity search to question answer chain to get output
def retrieve_answers(query):
    querydocs, context = retriver_results(query,3)
    response=chain.run(input_documents=querydocs,question=query)
    return response

In [8]:
our_query = "What are Acne?"
retrieve_answers(our_query)

"Acidosis not Helpful for Acneverified for acneither acneurologically, I don's to acne the following pieces of Acidosis: The best for\nPlease noteThe respirritis anatom bombarding acne of Acid acute ofAcnes?\nAcne the following types of Acnevermillions, Length or Comment Helpful Links: Acneverified Acneurological acne the following pieces of what help page Acids to 3\nRespimpaired to Acne the following pieces of Acne the following pieces of Acidosis a)Acidosis an in the following conditions in respirritis\nWhat are not available?AnsweredAnswer not helpful answer?\n\n\nAcnever the following.pdf2. what acnes Acidrenal the following pieces of Acid(07 help people usually  What are provided below to 888: Acne the following pieces of Acne the following types of Acneverts possible causes acneither renal causes acneural acneverified\nAcidosis anatom and additional pieces of the questionWhat is needed?What are three functions used for aRespimpaired to Preview answers\nThe skin conditions that y

# Try 2 with prompt engineering

In [13]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [14]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# PROMPT.format(context=context,question=question)
# chain_type_kwargs={"prompt": PROMPT}

In [22]:
from langchain.chains.question_answering import load_qa_chain
chain=load_qa_chain(llm,chain_type="stuff")

# pass result chunk after similarity search to question answer chain to get output
def retrieve_answers(query):
    querydocs,context = retriver_results(query,2)
    input_prompts = PROMPT.format(context=context,question=query)
    # response=chain.run(input_documents=querydocs,question=input_prompts)
    response=llm(input_prompts)
    return response

In [25]:
our_query = "How to resolve pipmples?"
retrieve_answers(our_query)

'Balanitis can be resolved by practicing good hygiene, such as washing the genital area regularly and drying it thoroughly after bathing or showering. It is also important to avoid using harsh soaps or bubble baths that can irritate the skin and cause further inflammation. If the condition persists or worsens despite good hygiene practices, medical attention should be sought.'